In [15]:
url = "https://www.nhs.uk/start-for-life/baby/recipes-and-meal-ideas/"

In [16]:
import trafilatura

In [17]:
html = trafilatura.fetch_url(url)

In [18]:
text = trafilatura.extract(html)

In [19]:
text

'Recipes and meal ideas\nFrom first tastes and finger foods to breakfast, lunch and dinner, check out our recipes for babies of all ages.\nAll recipes\n97 results\nDid you know\nBabies under 12 months do not need snacks. If you think your baby is hungry in between meals, offer extra milk feeds instead.\nVideo: First food and drinks\nHear from other parents in this video about how they started weaning their babies.'

In [48]:
breakfast_url = "https://www.nhs.uk/start-for-life/baby/recipes-and-meal-ideas/breakfast-cups/"

In [49]:
html=trafilatura.fetch_url(breakfast_url)

In [50]:
text=trafilatura.extract(html)
text

'Breakfast cups recipe\nOven-baked eggs, mushrooms and parsley served with toast and cooked tomatoes. This quick and easy recipe can be enjoyed for breakfast or lunch.\nAge: 10 to 12 months\nPrep: 10 mins\nCook: 15 mins\nPortions: 8\nIngredients\n- 5 medium eggs\n- 4 or 5 mushrooms\n- 5g fresh parsley (1 tbsp, chopped)\n- 3 cherry tomatoes (1 portion)\n- 1 small slice wholemeal bread (1 portion)\n- 1 tsp vegetable oil spread (1 portion)\nMethod\nPreheat the oven to 180C, fan 160C, gas mark 4. Beat the egg. Roughly chop the mushrooms and parsley.\nAdd the mushrooms to the egg and divide mixture evenly between 8 non-stick (or lightly oiled) muffin tray holes. Sprinkle each cup with parsley.\nBake for 10 to 15 minutes, or until golden, and the eggs are cooked through (and not runny).\nMeanwhile, put 2 tbsp water and the quartered cherry tomatoes in a small saucepan. Heat gently for 2 to 3 minutes, stirring often, until the tomatoes are soft (if the tomatoes start sticking, add a little wa

In [ ]:
# again need bs4 for the url extraction from "https://www.nhs.uk/start-for-life/baby/recipes-and-meal-ideas/"

In [20]:
import requests
from bs4 import BeautifulSoup

In [22]:
response = requests.get(url)
html = response.text
# html

In [25]:
soup = BeautifulSoup(html, "html.parser")
# soup

In [42]:
links = [a.get("href") for a in soup.find_all("a") if a.get("href") ]
# links

In [43]:
relevant_links = [rl for rl in links if rl.startswith("/start-for-life/baby/recipes-and-meal-ideas/")]
# relevant_links

In [ ]:
for rl in relevant_links:
    print(rl.strip("/").split("/")[-1].replace("-", "_"))

In [46]:
import re

In [54]:
lines = text.split("\n")
lines

['Breakfast cups recipe',
 'Oven-baked eggs, mushrooms and parsley served with toast and cooked tomatoes. This quick and easy recipe can be enjoyed for breakfast or lunch.',
 'Age: 10 to 12 months',
 'Prep: 10 mins',
 'Cook: 15 mins',
 'Portions: 8',
 'Ingredients',
 '- 5 medium eggs',
 '- 4 or 5 mushrooms',
 '- 5g fresh parsley (1 tbsp, chopped)',
 '- 3 cherry tomatoes (1 portion)',
 '- 1 small slice wholemeal bread (1 portion)',
 '- 1 tsp vegetable oil spread (1 portion)',
 'Method',
 'Preheat the oven to 180C, fan 160C, gas mark 4. Beat the egg. Roughly chop the mushrooms and parsley.',
 'Add the mushrooms to the egg and divide mixture evenly between 8 non-stick (or lightly oiled) muffin tray holes. Sprinkle each cup with parsley.',
 'Bake for 10 to 15 minutes, or until golden, and the eggs are cooked through (and not runny).',
 'Meanwhile, put 2 tbsp water and the quartered cherry tomatoes in a small saucepan. Heat gently for 2 to 3 minutes, stirring often, until the tomatoes are s

In [56]:
title = lines[0].replace("recipe", "").strip().title()
title

'Breakfast Cups'

In [62]:
description = lines[1].strip()
description

'Oven-baked eggs, mushrooms and parsley served with toast and cooked tomatoes. This quick and easy recipe can be enjoyed for breakfast or lunch.'

In [63]:
metadata = {
    "age_range": None,
    "prep_time": None,
    "cook_time": None,
    "portions": None
}

In [72]:
for line in lines:
    if line.lower().startswith("age:"):
        metadata["age_range"] = line.split(":", 1)[1].strip()
    elif line.lower().startswith("prep:"):
        metadata["prep_time"] = line.split(":", 1)[1].strip()
    elif line.lower().startswith("cook:"):
        metadata["cook_time"] = line.split(":", 1)[1].strip()
    elif line.lower().startswith("portions:"):
        portions_text = line.split(":", 1)[1].strip()
        metadata["portions"] = int(re.search(r"\d+", portions_text).group())
        
        
# `l.strip().split(":")[1]` and `l.split(":", 1)[1].strip()`  2nd is correct
# Splits the original line, but only at the first colon (maxsplit=1)
# Takes the second half ([1])
# Then strips just the value part
    
# the regex part in the portion is to deal with cases like 
# "8 small portions" or "about 4 servings", it will still get us 8 or 4

8


In [73]:
title, description, metadata

('Breakfast Cups',
 'Oven-baked eggs, mushrooms and parsley served with toast and cooked tomatoes. This quick and easy recipe can be enjoyed for breakfast or lunch.',
 {'age_range': '10 to 12 months',
  'prep_time': '10 mins',
  'cook_time': '15 mins',
  'portions': 8})

In [78]:
ingredients = []
in_ingredients_section = False

In [79]:
for line in lines:
    if line.strip().lower() == "ingredients":
        in_ingredients_section = True
        continue
    if in_ingredients_section:
        if line.strip().lower().startswith("method"):
            break  # Stop when we reach the next section
        if line.strip().startswith("-"):
            ingredients.append(line.strip().lstrip("-").strip())
            
# if stripped and not stripped.startswith("-") and stripped[0].isupper() and len(stripped.split()) <= 4:            
# This checks if the line is likely a new section heading:
# - not empty
# - doesn't start with a bullet (-)
# - starts with a capital letter
# - has 4 or fewer words
# If all true, we treat it as the start of a new section (e.g., "Method", "Weaning tip", etc.)

In [80]:
ingredients

['5 medium eggs',
 '4 or 5 mushrooms',
 '5g fresh parsley (1 tbsp, chopped)',
 '3 cherry tomatoes (1 portion)',
 '1 small slice wholemeal bread (1 portion)',
 '1 tsp vegetable oil spread (1 portion)']

In [81]:
text = """
Weaning tip
From 10 to 12 months, your child should be having 3 meals a day.
More recipes
More in recipes and meal ideas
"""

lines = text.strip().split("\n")
weaning_tip_next = False
weaning_tip = None

for line in lines:
    stripped = line.strip()
    if stripped.lower() == "weaning tip":
        weaning_tip_next = True
        continue
    elif weaning_tip_next and stripped:
        weaning_tip = stripped
        break  # stop here, no need to keep going

print("Captured tip:", weaning_tip)


Captured tip: From 10 to 12 months, your child should be having 3 meals a day.
